# Processing data from GHTorrent

### PLEASE DO NOT RUN THIS CODE - ONLY for reproduceability
The processed repos are already saved under *data/ghtorrent_python_notdeleted_repos_updated2019*

We process the mysql files from GHTorrent using a python script.

We get only the following tables:
   - projects
   - project_languages
   - repo_labels
   - watchers

we get all repositories from the table 'projects' with the following:
   - language=Python
   - Not deleted
   - Not forked from any other repo
   - has a url and name 
   - last updated in 2019
   
The data is saved in a local postgres database and then copied as csv under *data/ghtorrent_python_notdeleted_repos_updated2019*.

In [1]:
import psycopg2
import pandas as pd

import numpy as np

In [2]:
# DOWNLOAD mysql-2019-06-01.tar from GHTORRENT:
# http://ghtorrent-downloads.ewi.tudelft.nl/mysql/mysql-2019-06-01.tar.gz
# then extract them and replace below with path for all csv files
path = "data/mysql-2019-06-01.tar/mysql-2019-06-01/{}.csv"


In [162]:
def insert_to_ghtorrent_db(table, rows):
    """ insert multiple rows into the vendors table  """
    insert_sql = '''INSERT INTO public.{}({}) VALUES ({});'''.format(table,','.join(table_columns[table]['columns']),
                                                                           ','.join(['%s']*len(table_columns[table]['columns'])))
    conn = None
    try:
        # Please define here a local database to use for insterting the data
        conn = psycopg2.connect(database="ghtorrent_restore",
                                user="ghtorrentuser",
                                password="ghtorrentpassword")
        # create a new cursor
        cur = conn.cursor()
        cur.executemany(insert_sql,rows)
        conn.commit()
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print("error while inserting")
        print(error)
    finally:
        if conn is not None:
            conn.close()

In [163]:

table_columns = {
    'projects':{ 'columns':['id', 'url', 'owner_id', 'name', 'description', 'language',
         'created_at', 'forked_from', 'deleted', 'updated_at', 'forked_commit_id'],
               'filter_on': ''}, #787034  
    'project_languages': {'columns':['project_id', 'language', 'bytes', 'created_at'],
               'filter_on': 'project_id'},
    'repo_labels': {'columns':['id', 'repo_id', 'name'],
               'filter_on': 'repo_id'},
    'watchers': {'columns':['repo_id', 'user_id', 'created_at'],
               'filter_on': 'repo_id'},
}


# we get all repositories:
#   - Python
#   - Not deleted
#   - Not forked from any other repo
#   - has a url and name 
#   - last updated in 2019

def filter_data(table, data, filter_on=None):
    if table == 'projects':
        languages = [ 'Python' ]
        data = data[(data['language'].isin(languages)) & 
                      (data['deleted'] == '0') & (data['forked_from'].isna()) & 
                      (data['url'].notna()) & (data['name'].notna()) & (data['name'].notna()) &
                      (data['updated_at'].str.startswith('2019')) ]
    elif filter_on is not None:
        project_ids = (pd.read_csv(path.format('projects_ids'), names=['ids'],
                                   na_values="\\N"))['ids'].values.tolist()
        data = data[data[filter_on].isin(project_ids) ]
        
        
    return data
def insert_chunks_ghtorrent(table, chunksize = 50000):
    i = 0
   
    
    for chunk in pd.read_csv(path.format(table), names=(table_columns[table])['columns'], 
                             chunksize=chunksize, na_values="\\N"):
        
        
        chunk = filter_data(table, chunk, filter_on = (table_columns[table])['filter_on'])
        chunk = chunk.replace({np.nan: None})

        print('adding ', len(chunk), ' rows ')
        i += len(chunk)
        if len(chunk) > 0:
            insert_to_ghtorrent_db(table, chunk.values.tolist())

    print('added a total of ', i, ' rows ')
    
    
def insert_tables_ghtorrent():
    for t in table_columns.keys():
        if t != 'projects':
            print('processing table', t)
            insert_chunks_ghtorrent(t)

In [164]:
insert_tables_ghtorrent()

processing table project_languages
adding  204  rows 
adding  137  rows 
adding  212  rows 
adding  115  rows 
adding  128  rows 
adding  142  rows 
adding  135  rows 
adding  120  rows 
adding  227  rows 
adding  168  rows 
adding  127  rows 
adding  192  rows 
adding  209  rows 
adding  128  rows 
adding  142  rows 
adding  164  rows 
adding  177  rows 
adding  117  rows 
adding  180  rows 
adding  141  rows 
adding  218  rows 
adding  190  rows 
adding  90  rows 
adding  136  rows 
adding  127  rows 
adding  185  rows 
adding  159  rows 
adding  133  rows 
adding  227  rows 
adding  143  rows 
adding  108  rows 
adding  171  rows 
adding  193  rows 
adding  90  rows 
adding  92  rows 
adding  121  rows 
adding  150  rows 
adding  125  rows 
adding  152  rows 
adding  181  rows 
adding  140  rows 
adding  155  rows 
adding  176  rows 
adding  208  rows 
adding  85  rows 
adding  143  rows 
adding  153  rows 
adding  214  rows 
adding  120  rows 
adding  21  rows 
adding  241  rows 
a

adding  214  rows 
adding  565  rows 
adding  220  rows 
adding  126  rows 
adding  138  rows 
adding  162  rows 
adding  177  rows 
adding  242  rows 
adding  197  rows 
adding  245  rows 
adding  150  rows 
adding  1204  rows 
adding  136  rows 
adding  178  rows 
adding  152  rows 
adding  278  rows 
adding  122  rows 
adding  78  rows 
adding  157  rows 
adding  204  rows 
adding  90  rows 
adding  186  rows 
adding  298  rows 
adding  241  rows 
adding  222  rows 
adding  334  rows 
adding  232  rows 
adding  145  rows 
adding  136  rows 
adding  166  rows 
adding  151  rows 
adding  176  rows 
adding  163  rows 
adding  259  rows 
adding  143  rows 
adding  136  rows 
adding  107  rows 
adding  366  rows 
adding  148  rows 
adding  225  rows 
adding  118  rows 
adding  217  rows 
adding  127  rows 
adding  128  rows 
adding  98  rows 
adding  238  rows 
adding  232  rows 
adding  191  rows 
adding  130  rows 
adding  100  rows 
adding  194  rows 
adding  214  rows 
adding  167  r

adding  156  rows 
adding  151  rows 
adding  228  rows 
adding  221  rows 
adding  343  rows 
adding  366  rows 
adding  182  rows 
adding  152  rows 
adding  143  rows 
adding  36  rows 
adding  152  rows 
adding  299  rows 
adding  284  rows 
adding  174  rows 
adding  492  rows 
adding  145  rows 
adding  248  rows 
adding  180  rows 
adding  194  rows 
adding  157  rows 
adding  184  rows 
adding  150  rows 
adding  74  rows 
adding  173  rows 
adding  286  rows 
adding  129  rows 
adding  155  rows 
adding  138  rows 
adding  112  rows 
adding  270  rows 
adding  216  rows 
adding  163  rows 
adding  69  rows 
adding  153  rows 
adding  161  rows 
adding  135  rows 
adding  197  rows 
adding  161  rows 
adding  257  rows 
adding  159  rows 
adding  112  rows 
adding  146  rows 
adding  171  rows 
adding  89  rows 
adding  167  rows 
adding  125  rows 
adding  204  rows 
adding  171  rows 
adding  184  rows 
adding  134  rows 
adding  228  rows 
adding  142  rows 
adding  145  row

adding  195  rows 
adding  208  rows 
adding  265  rows 
adding  177  rows 
adding  229  rows 
adding  208  rows 
adding  275  rows 
adding  205  rows 
adding  403  rows 
adding  151  rows 
adding  269  rows 
adding  275  rows 
adding  381  rows 
adding  241  rows 
adding  148  rows 
adding  74  rows 
adding  184  rows 
adding  140  rows 
adding  179  rows 
adding  151  rows 
adding  136  rows 
adding  140  rows 
adding  203  rows 
adding  192  rows 
adding  182  rows 
adding  208  rows 
adding  211  rows 
adding  181  rows 
adding  226  rows 
adding  57  rows 
adding  174  rows 
adding  180  rows 
adding  155  rows 
adding  157  rows 
adding  146  rows 
adding  210  rows 
adding  198  rows 
adding  339  rows 
adding  147  rows 
adding  143  rows 
adding  132  rows 
adding  180  rows 
adding  100  rows 
adding  138  rows 
adding  173  rows 
adding  124  rows 
adding  164  rows 
adding  226  rows 
adding  141  rows 
adding  86  rows 
adding  173  rows 
adding  120  rows 
adding  105  ro

adding  1  rows 
adding  0  rows 
adding  26  rows 
adding  1  rows 
adding  8  rows 
adding  2  rows 
adding  5  rows 
adding  0  rows 
adding  2  rows 
adding  1  rows 
adding  3  rows 
adding  36  rows 
adding  1  rows 
adding  0  rows 
adding  0  rows 
adding  3  rows 
adding  1  rows 
adding  2  rows 
adding  12  rows 
adding  15  rows 
adding  1  rows 
adding  1  rows 
adding  12  rows 
adding  0  rows 
adding  8  rows 
adding  0  rows 
adding  2  rows 
adding  0  rows 
adding  2  rows 
adding  0  rows 
adding  3  rows 
adding  3  rows 
adding  17  rows 
adding  1  rows 
adding  1  rows 
adding  0  rows 
adding  0  rows 
adding  0  rows 
adding  2  rows 
adding  2  rows 
adding  0  rows 
adding  9  rows 
adding  14  rows 
adding  19  rows 
adding  15  rows 
adding  24  rows 
adding  26  rows 
adding  16  rows 
adding  40  rows 
adding  42  rows 
adding  142  rows 
adding  116  rows 
adding  110  rows 
adding  124  rows 
adding  130  rows 
adding  103  rows 
adding  116  rows 
add

...

adding  2424  rows 
adding  2096  rows 
adding  5753  rows 
adding  1729  rows 
adding  4614  rows 
adding  10135  rows 
adding  2433  rows 
adding  5239  rows 
adding  12597  rows 
adding  3504  rows 
adding  2422  rows 
adding  1722  rows 
adding  1274  rows 
adding  3494  rows 
adding  1930  rows 
adding  826  rows 
adding  2104  rows 
adding  568  rows 
adding  1286  rows 
adding  8507  rows 
adding  5428  rows 
adding  120  rows 
adding  45  rows 
adding  296  rows 
adding  3356  rows 
adding  9185  rows 
adding  9812  rows 
adding  21387  rows 
adding  481  rows 
adding  2030  rows 
adding  5267  rows 
adding  1083  rows 
adding  2065  rows 
adding  3645  rows 
adding  3162  rows 
adding  2221  rows 
adding  3318  rows 
adding  1552  rows 
adding  2543  rows 
adding  5638  rows 
adding  1269  rows 
adding  710  rows 
adding  9789  rows 
adding  37872  rows 
adding  1743  rows 
adding  4625  rows 
adding  5412  rows 
adding  1227  rows 
adding  2198  rows 
adding  3776  rows 
addi

adding  3293  rows 
adding  618  rows 
adding  737  rows 
adding  7213  rows 
adding  2568  rows 
adding  1255  rows 
adding  2610  rows 
adding  1476  rows 
adding  2219  rows 
adding  860  rows 
adding  2512  rows 
adding  2739  rows 
adding  3138  rows 
adding  8599  rows 
adding  4705  rows 
adding  3308  rows 
adding  2031  rows 
adding  1813  rows 
adding  1602  rows 
adding  1881  rows 
adding  2335  rows 
adding  625  rows 
adding  8866  rows 
adding  2057  rows 
adding  3036  rows 
adding  18250  rows 
adding  6120  rows 
adding  1402  rows 
adding  11076  rows 
adding  1959  rows 
adding  57  rows 
adding  4836  rows 
adding  3976  rows 
adding  2191  rows 
adding  8219  rows 
adding  808  rows 
adding  1388  rows 
adding  2541  rows 
adding  2729  rows 
adding  1512  rows 
adding  2697  rows 
adding  15286  rows 
adding  2583  rows 
adding  8195  rows 
adding  2130  rows 
adding  5381  rows 
adding  8279  rows 
adding  682  rows 
adding  367  rows 
adding  2808  rows 
adding

adding  1386  rows 
adding  1807  rows 
adding  663  rows 
adding  603  rows 
adding  8855  rows 
adding  1021  rows 
adding  4875  rows 
adding  1268  rows 
adding  1598  rows 
adding  1285  rows 
adding  1373  rows 
adding  1983  rows 
adding  6846  rows 
adding  542  rows 
adding  156  rows 
adding  7349  rows 
adding  2014  rows 
adding  86  rows 
adding  538  rows 
adding  337  rows 
adding  891  rows 
adding  359  rows 
adding  781  rows 
adding  359  rows 
adding  366  rows 
adding  675  rows 
adding  1002  rows 
adding  1476  rows 
adding  372  rows 
adding  560  rows 
adding  675  rows 
adding  3810  rows 
adding  1946  rows 
adding  1975  rows 
adding  11085  rows 
adding  850  rows 
adding  422  rows 
adding  2163  rows 
adding  3895  rows 
adding  756  rows 
adding  2092  rows 
adding  6022  rows 
adding  3977  rows 
adding  2844  rows 
adding  4317  rows 
adding  854  rows 
adding  9131  rows 
adding  1513  rows 
adding  4912  rows 
adding  1162  rows 
adding  2922  rows 


adding  7345  rows 
adding  5333  rows 
adding  1974  rows 
adding  1607  rows 
adding  7850  rows 
adding  2851  rows 
adding  366  rows 
adding  1636  rows 
adding  110  rows 
adding  571  rows 
adding  895  rows 
adding  6263  rows 
adding  654  rows 
adding  2232  rows 
adding  1139  rows 
adding  1017  rows 
adding  461  rows 
adding  3654  rows 
adding  4258  rows 
adding  2521  rows 
adding  1359  rows 
adding  1209  rows 
adding  1875  rows 
adding  2375  rows 
adding  662  rows 
adding  533  rows 
adding  1735  rows 
adding  3252  rows 
adding  3631  rows 
adding  367  rows 
adding  3129  rows 
adding  3168  rows 
adding  1669  rows 
adding  4602  rows 
adding  855  rows 
adding  5263  rows 
adding  258  rows 
adding  217  rows 
adding  780  rows 
adding  515  rows 
adding  1507  rows 
adding  1476  rows 
adding  599  rows 
adding  1127  rows 
adding  6332  rows 
adding  1865  rows 
adding  1615  rows 
adding  4566  rows 
adding  982  rows 
adding  2273  rows 
adding  1339  ro

adding  3251  rows 
adding  4894  rows 
adding  4946  rows 
adding  3085  rows 
adding  3003  rows 
adding  1994  rows 
adding  2541  rows 
adding  4714  rows 
adding  5566  rows 
adding  2942  rows 
adding  3656  rows 
adding  5975  rows 
adding  4125  rows 
adding  4502  rows 
adding  8036  rows 
adding  5694  rows 
adding  5840  rows 
adding  6107  rows 
adding  9249  rows 
adding  6186  rows 
adding  4644  rows 
adding  3804  rows 
adding  8443  rows 
adding  4835  rows 
adding  5248  rows 
adding  3616  rows 
adding  6691  rows 
adding  7119  rows 
adding  3036  rows 
adding  10058  rows 
adding  6193  rows 
adding  6840  rows 
adding  4439  rows 
adding  4082  rows 
adding  5443  rows 
adding  5135  rows 
adding  9047  rows 
adding  4836  rows 
adding  0  rows 
adding  0  rows 
adding  0  rows 
adding  0  rows 
adding  753  rows 
adding  5276  rows 
adding  2094  rows 
adding  4146  rows 
adding  5007  rows 
adding  4340  rows 
adding  2985  rows 
adding  3661  rows 
adding  4858

In [15]:
watchers['watchers'].value_counts()

0       423045
1        66202
2        23286
3        13036
4         9205
         ...  
4267         1
5465         1
1371         1
680          1
2295         1
Name: watchers, Length: 2158, dtype: int64